# 실습1
## SBS 뉴스데이터 수집하기

In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
news_rss = requests.get('https://news.sbs.co.kr/news/TopicRssFeed.do?plink=RSSREADER')
news_rss_soup = BeautifulSoup(news_rss.content, 'xml')
link_list = news_rss_soup.select('item > link')
print("기사 개수: ", len(link_list))

기사 개수:  8


In [3]:
title_list = news_rss_soup.select('item > title')
title_list = [title.text for title in title_list]

news_data = []

# 기사 본문 수집
for link in link_list:
    news_response = requests.get(link.text)
    news_content_soup = BeautifulSoup(news_response.content, 'html.parser')
    news_content = news_content_soup.select_one("div[itemprop=articleBody]")

    if news_content:
        # 텍스트만 추출하고 특수기호 디코딩
        clean_text = news_content.get_text(separator=' ', strip=True)
        clean_text = clean_text.replace('▲', '').strip() # 특수기호 제거
    else:
        clean_text = ""
        
    news_data.append(clean_text)
    
import pandas as pd
news_df = pd.DataFrame(data={'title': title_list, 'content': news_data})
news_df.to_csv("news.csv", encoding="utf-8-sig", index=False)
print("Save complete")

Save complete


## db에 연결하기

In [4]:
import pymysql

# DB 연결
conn = pymysql.connect(host='localhost', user='root', password='1234', db='web_data', charset='utf8')
cursor = conn.cursor()

# 테이블 생성
create_table_sql = '''
CREATE TABLE IF NOT EXISTS news (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title TEXT,
    content TEXT
)
'''
cursor.execute(create_table_sql)
conn.commit()

In [5]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('news.csv')

# INSERT 쿼리 정의
insert_sql = '''
INSERT INTO news (title, content)
VALUES (%s, %s)
'''

# 행 단위 삽입
for _, row in df.iterrows():
    cursor.execute(insert_sql, (row['title'], row['content']))

conn.commit()
print("✔ DB 저장 완료")

# 연결 종료
cursor.close()
conn.close()

✔ DB 저장 완료
